In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTING ALL LIBRARIES

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectPercentile
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from scipy.stats import chi2_contingency
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import MinMaxScaler
from textblob.sentiments import NaiveBayesAnalyzer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")





# LOADING DATASET

In [ ]:
train=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/train.csv (1)/train.csv')
test=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv')
sample=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/sample (1).csv')
movie_data=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/movies.csv (1)/movies.csv')

# DUMMY SUBMISSION

In [ ]:
'''
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train["sentiment"])

X = train["movieid"]
y = le.transform(train["sentiment"])
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(X)
sub = pd.DataFrame(le.inverse_transform(dummy_clf.predict(test_data["movieid"])), columns=['sentiment'])
sub.index.name = 'id'
sub.to_csv("submission.csv", encoding='utf-8')

output = pd.read_csv("submission.csv")"""

'''

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
movie_data.head()

In [ ]:
train= train.dropna()
train= train.drop_duplicates()
movie_data=movie_data.dropna()



In [ ]:
import string

def preprocess_text(text):
    # Remove special characters and punctuations
    text = ''.join([char if char not in string.punctuation else ' ' for char in text])
    # Remove extra white spaces
    text = ' '.join(text.split())
    return text

train['reviewText'] = train['reviewText'].astype(str)
test['reviewText'] = test['reviewText'].astype(str)

train['reviewText'] = train['reviewText'].apply(preprocess_text)
test['reviewText'] = test['reviewText'].apply(preprocess_text)




In [ ]:
movie_data = movie_data.drop_duplicates(subset='movieid')

# MERGE MOVIE DATASET

In [ ]:
train = pd.merge(train, movie_data[['movieid','title','releaseDateTheaters','releaseDateStreaming','director', 'audienceScore','rating','soundType','boxOffice','ratingContents','runtimeMinutes','genre','originalLanguage']], on='movieid', how='left')
test=pd.merge(test,movie_data[['movieid','director','audienceScore','genre','releaseDateTheaters','releaseDateStreaming','rating','boxOffice','soundType','title','ratingContents','originalLanguage','runtimeMinutes']],on='movieid',how='left')

train.shape,test.shape

# PREPROCESSING

In [ ]:
train['reviewText'] = train['reviewText'].str.lower()
test['reviewText']=test['reviewText'].str.lower()

In [ ]:
numCols = list(train.select_dtypes("number").columns)
catCols = list(train.select_dtypes(exclude = "number").columns)
print('category_columns:',catCols)
print('numerical_columns:',numCols)


In [ ]:
genre_values = train['genre'].unique()
print(f"genre has {len(genre_values)} ")
train['genre'].value_counts()

In [ ]:
#  one-hot encoding on the 'rating' column
df_encoded = pd.get_dummies(train, columns=['rating'], prefix='rating')

# Plot of distribution of ratings
plt.figure(figsize=(6, 4))
sns.countplot(data=df_encoded, x='rating_PG-13', color='red', label='PG-13')
sns.countplot(data=df_encoded, x='rating_R', color='orange', label='R')
sns.countplot(data=df_encoded, x='rating_PG', color='green', label='PG')
plt.title('Distribution of Rating')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.legend() 
plt.show()


**AUDIENCE SCORE PLOT**

In [ ]:


# boxplot
sns.boxplot(x='sentiment', y='audienceScore', data=train)

plt.title("Sentiment Distribution based on Audience Score")
plt.xlabel("Sentiment")
plt.ylabel("Audience Score")

# Display the plot
plt.show()

# CATEGORICAL FEATURES

In [ ]:
def fill_nan_with_empty_string(df, columns):
    for column in columns:
        df[column] = df[column].fillna('').astype(str)
    return df

#  columns  to process
columns_to_process = ['director', 'genre', 'title',  'ratingContents', 'rating']

# Apply the function to both test and train 
test = fill_nan_with_empty_string(test, columns_to_process)
train = fill_nan_with_empty_string(train, columns_to_process)

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
train.dropna()

In [ ]:
test.isna().sum()

In [ ]:
train.isna().sum()


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='sentiment', data=train)
plt.title("Distribution of Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("Count")
plt.show()

**Dolby Digital is the most frequet Sound Type**

In [ ]:

def plot_sound_type_distribution_pie(data, threshold=0.05):
   
    # Count the occurrences of each sound type
    sound_type_counts = train['soundType'].value_counts()

   
    total_count = sound_type_counts.sum()

    # Filter categories based on the threshold
    significant_categories = sound_type_counts[sound_type_counts / total_count > threshold]
    other_count = total_count - significant_categories.sum()

    # labels for significant categories and 'Other' category
    labels = significant_categories.index.tolist() + ['Other']
    counts = significant_categories.tolist() + [other_count]


    plt.figure(figsize=(8, 8))
    plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Sound Type Distribution')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()


plot_sound_type_distribution_pie(train['soundType'])


**ORIGINAL LANGUAGE DISTRIBUTION**

In [ ]:
import matplotlib.pyplot as plt

def plot_original_language_distribution(data):
    
   
    language_counts = data.value_counts()

    # Create a bar plot
    plt.figure(figsize=(18, 6))
    language_counts.plot(kind='bar')
    plt.title('Original Language Distribution')
    plt.xlabel('Original Language')
    plt.ylabel('Count')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.show()

# Assuming you have a DataFrame 'df' with an 'originalLanguage' column
plot_original_language_distribution(train['originalLanguage'])


# NUMERICAL FEATURES

In [ ]:

def convert_numeric(value):
    value = value.replace('$', '')
    if value.endswith('M'):
        return float(value.replace('M', '')) * 1000000
    elif value.endswith('K'):
        return float(value.replace('K', '')) * 1000
    elif value == '':
        return 0  
    else:
        return float(value)


def convert_boxoffice_to_numeric(df):
    df['boxOffice'] = df['boxOffice'].astype(str).apply(convert_numeric)
    return df


train = convert_boxoffice_to_numeric(train)


test = convert_boxoffice_to_numeric(test)




In [ ]:


numeric_columns = ['boxOffice', 'audienceScore', 'runtimeMinutes']

# Step 1: Convert numeric columns to numeric format
train[numeric_columns] = train[numeric_columns].apply(pd.to_numeric, errors='coerce')
test[numeric_columns] = test[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Step 2: Handle missing or invalid values by filling with column means
train[numeric_columns] = train[numeric_columns].fillna(train[numeric_columns].mean())
test[numeric_columns] = test[numeric_columns].fillna(test[numeric_columns].mean())




In [ ]:



plt.figure(figsize=(12, 8))

# Plot the histogram with KDE
sns.histplot(train['boxOffice'], bins=20, kde=True, color='red')

# Add a title and labels
plt.title("Distribution of Box Office", fontsize=16)
plt.xlabel("Box Office", fontsize=14)
plt.ylabel("Frequency", fontsize=14)

# Set the y-axis limit
plt.ylim(0, 10000)

# Show the plot
plt.show()



In [ ]:


plt.figure(figsize=(10, 6))
plt.hist(train['audienceScore'], bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Audience Score')
plt.ylabel('Frequency')
plt.title('Distribution of Audience Scores')
plt.show()

In [ ]:
# Normalize numeric columns using MinMaxScaler


scaler = MinMaxScaler()
train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])




In [ ]:
# Fill missing values in 'soundType' with the mode
train['soundType'] = train['soundType'].fillna(train['soundType'].mode()[0])
test['soundType'] = test['soundType'].fillna(test['soundType'].mode()[0])

In [ ]:
# Fill missing values in 'originalLanguage' with the mode
train['originalLanguage'] = train['originalLanguage'].fillna(train['originalLanguage'].mode()[0])
test['originalLanguage'] = test['originalLanguage'].fillna(test['originalLanguage'].mode()[0])

In [ ]:
train.isna().sum()

In [ ]:
'''
train['audienceScoreBoxOffice'] = train['audienceScore'] * train['boxOffice']
test['audienceScoreBoxOffice'] = test['audienceScore'] * test['boxOffice']
'''

# TRAIN TEST SPLIT

In [ ]:
X = train[['reviewText','audienceScore','soundType','originalLanguage','boxOffice','rating','genre','director','title','runtimeMinutes','movieid','ratingContents','reviewerName']]
y=train['sentiment']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

**Chi squared scores**

**reviewText                0.265843,
genre                    19.714405,
director                   4.94023,
title                     3.144161,
movieid                   2.639399,
ratingContents            4.788659,
reviewerName               1.93519,
originalLanguage         10.688852,
soundType                52.567118**

In [ ]:
'''
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2


features = ['reviewText', 'genre', 'director', 'title', 'movieid', 'ratingContents', 'reviewerName', 'originalLanguage', 'soundType']

# Initialize a DataFrame to store the mutual information scores
M_scores = pd.DataFrame(index=features, columns=['Mutual_Info_Score'])

# Iterate through each text feature
for feature in features:
    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.2)
    
    # Fit and transform the text data for the current feature
    X_text = vectorizer.fit_transform(X_train[feature])
    
    # Calculate the chi-squared statistics and p-values
    chi2_stats, p_values = chi2(X_text, y_train)
    
    # Store the mean of the chi-squared statistics as the mutual information score
    chi2_score = chi2_stats.mean()
    
    # Store the mutual information score in the DataFrame
    M_scores.loc[feature, 'Mutual_Info_Score'] = chi2_score

# Print the mutual information scores
print("Chi-Squared Scores:")
print(M_scores)
'''

# MODELS

# SGD

 **Best Hyperparameters**: **{'classifier__alpha': 0.0001, 'classifier__penalty': 'l2'},**
 **F1 Score**: **0.8318247187209203,**
 **Validation Accuracy**: **0.7438886920673774**

In [ ]:
'''
from sklearn.linear_model import SGDClassifier


column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])
# pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', SGDClassifier(max_iter=1000, loss='log', random_state=42))
])

#hyperparameters 
param_grid = {
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__penalty': ['l2', 'l1', 'elasticnet']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Predict labels for the test data using the best estimator
y_pred = best_estimator.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred)

# Calculate the F1 score
f1 = f1_score(y_true_encoded, y_pred_encoded)

# Print the best hyperparameters and F1 score
print("Best Hyperparameters:", grid_search.best_params_)
print("F1 Score:", f1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)
'''

# LOGISTIC REGRESSION

**F1 Score**: **0.885122940252678**,
**Validation Accuracy**: **0.8421527263357919** ,
**Best Hyperparameters**: **{'classifier__C': 10}**

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.79      0.71      0.75     10261
    POSITIVE       0.86      0.91      0.89     20788

    accuracy                           0.84     31049
   macro avg       0.83      0.81      0.82     31049
weighted avg       0.84      0.84      0.84     31049

In [ ]:
'''

# Create the ColumnTransformer
column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])

# Create the pipeline with the best hyperparameters
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', LogisticRegression(max_iter=1000, solver='sag'))
])
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10]}
# Create the GridSearchCV 
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')



# Get the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Predict labels for the test data using the best estimator
y_pred = best_estimator.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred)

#  F1 score and accuracy
f1 = f1_score(y_true_encoded, y_pred_encoded, average='macro')
accuracy = accuracy_score(y_true_encoded, y_pred_encoded)

print("Best Hyperparameters:", grid_search.best_params_)
print("F1 Score:", f1)
print("Validation Accuracy:", accuracy)
class_report = classification_report(y_true_encoded, y_pred_encoded, target_names=label_encoder.classes_)

# Print the classification report
print("Classification Report:\n", class_report)

'''





# MLP CLASSIFIER

**F1 Score**: **0.8138773378332229**,
**Validation Accuracy**: **0.701600695674579**

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.74      0.15      0.25     10261
    POSITIVE       0.70      0.97      0.81     20788

    accuracy                           0.70     31049
   macro avg       0.72      0.56      0.53     31049
weighted avg       0.71      0.70      0.63     31049

In [ ]:
'''
column_transformer = ColumnTransformer([
    ('text_vectorizer', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_ratingContents', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'ratingContents'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes'])
])

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', MLPClassifier(hidden_layer_sizes=(5,10), activation='relu', solver='adam', random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict labels for the test data
y_pred_mlp = pipeline.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred_mlp) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred_mlp)

# Calculate the F1 score and accuracy for the MLP classifier
f1_mlp = f1_score(y_true_encoded, y_pred_encoded)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

# Print the results
print("MLP Classifier:")
print("F1 Score:", f1_mlp)
print("Validation Accuracy:", accuracy_mlp)
class_report = classification_report(y_true_encoded, y_pred_encoded, target_names=label_encoder.classes_)

# Print the classification report
print("Classification Report:\n", class_report)
'''



# LINEAR SVC
# (Feature Selection used)

**F1 Score**: **0.8875350367161621**,
**Validation Accuracy**: **0.8436342555315791**


Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.81      0.69      0.74     10261
    POSITIVE       0.86      0.92      0.89     20788

    accuracy                           0.84     31049
   macro avg       0.83      0.80      0.82     31049
weighted avg       0.84      0.84      0.84     31049




In [ ]:
'''
# Create the ColumnTransformer
column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])
# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
     ('feature_selector', SelectPercentile(percentile=38)),
    ('classifier', LinearSVC(C=0.4, penalty='l2', tol=0.001,verbose= False, max_iter= 1000))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict labels for the test data
y_pred = pipeline.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred)

# Calculate the F1 score
f1 = f1_score(y_true_encoded, y_pred_encoded)

# Print the F1 score
print("F1 Score:", f1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)
class_report = classification_report(y_true_encoded, y_pred_encoded, target_names=label_encoder.classes_)

# Print the classification report
print("Classification Report:\n", class_report)
'''



# KNEIBHBOUR CLASSIFIER

**F1 Score**: **0.8268080239310224** ,**Validation Accuracy:** **0.7464008502689298**


Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.69      0.43      0.53     10261
    POSITIVE       0.76      0.90      0.83     20788

    accuracy                           0.75     31049
   macro avg       0.72      0.67      0.68     31049
 weighted avg       0.74      0.75      0.73     31049

In [ ]:
'''


column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', MinMaxScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])


# Create a pipeline 
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', KNeighborsClassifier(n_neighbors=7))  # Set n_neighbors to 7
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict labels for the test data
y_pred = pipeline.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred)

# Calculate the F1 score
f1 = f1_score(y_true_encoded, y_pred_encoded)

# Print the F1 score
print("F1 Score:", f1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)
class_report = classification_report(y_true_encoded, y_pred_encoded, target_names=label_encoder.classes_)
# Print the classification report
'''


# MULTINOMIAL NB

**Best Hyperparameters**: **{'classifier__alpha': 0.1, 'classifier__fit_prior': False}**,
**F1 Score**:**0.8641420118343195**,
**Validation Accuracy**: **0.8151309220908886**

In [ ]:
'''
column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])
# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', MultinomialNB())
])

#hyperparameters 
param_grid = {
    'classifier__alpha': [0.1, 0.5, 1.0],
    'classifier__fit_prior': [True, False]
}

# Create GridSearchCV instance
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')

# Fit the grid search on the training data
grid_search.fit(X_train, y_train)

# Get the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Predict labels for the test data using the best estimator
y_pred = best_estimator.predict(X_test)

# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Encode the predicted labels (y_pred) using LabelEncoder
y_pred_encoded = label_encoder.transform(y_pred)

# Calculate the F1 score
f1 = f1_score(y_true_encoded, y_pred_encoded)

# Print the best hyperparameters and F1 score
print("Best Hyperparameters:", grid_search.best_params_)
print("F1 Score:", f1)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)"""

'''

# VOTING CLASSIFIER

In [ ]:




# Define the transformations for different columns
column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])
# Create an ensemble of classifiers using VotingClassifier
ensemble_clf = VotingClassifier(
    estimators=[
        ('linear_svc', LinearSVC(C=0.4, penalty='l2', tol=0.001, verbose=False, max_iter=1000)),
        ('logistic_regression', LogisticRegression(max_iter=1000, solver='sag', C=10))
    ],
    voting='hard' 
)

# Create a pipeline 
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('feature_selector', SelectPercentile(percentile=40)),
    ('classifier', ensemble_clf)
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict labels for the test data
y_pred = pipeline.predict(X_test)

# F1 score
f1 = f1_score(y_test, y_pred, average='micro')  


print("F1 Score:", f1)
label_encoder = LabelEncoder()
# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Validation Accuracy:", accuracy)
class_report = classification_report(y_test, y_pred)

# Print the classification report
print("Classification Report:\n", class_report)





# ROC CURVE (area=0.91)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Create the ColumnTransformer
column_transformer = ColumnTransformer([
    ('text_vectorizer_reviewText', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewText'),
    ('numeric_scaler', StandardScaler(), ['audienceScore', 'boxOffice', 'runtimeMinutes']),
    ('text_vectorizer_rating', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'rating'),
    ('text_vectorizer_title', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'title'),
    ('text_vectorizer_movied', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'movieid'),
    ('text_vectorizer_genre', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'genre'),
    ('text_vectorizer_reviewerName', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'reviewerName'),
    ('text_vectorizer_director', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'director'),
    ('text_vectorizer_soundType', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'soundType'),
    ('text_vectorizer_originalLanguage', TfidfVectorizer(ngram_range=(1, 2), max_df=0.2), 'originalLanguage')
])

# Create the pipeline with Logistic Regression
pipe = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', LogisticRegression(max_iter=1000, solver='sag', C=10, multi_class='ovr'))
])

# Fit the pipeline on training data
pipe.fit(X_train, y_train_encoded)



y_score = pipe.predict_proba(X_test)


# Encode the true labels (y_true) using LabelEncoder
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_test)

# Calculate the ROC curve for each class
fpr, tpr, _ = roc_curve(y_true_encoded, y_score[:, 1])  # Assuming 'positive' is class 1

# Calculate the AUC (Area Under the Curve)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# SUBMISSION

In [ ]:

X_test_data=test[['reviewText','audienceScore','soundType','boxOffice','rating','genre','director','title','originalLanguage','runtimeMinutes','movieid','ratingContents','reviewerName']] 
sub = pd.DataFrame(pipeline.predict(X_test_data), columns=['sentiment'])
sub.index.name = 'id'
sub.to_csv("submission.csv", encoding='utf-8')


In [ ]:
pd.read_csv('submission.csv')

In [ ]:
sentiment_summary = pd.read_csv('submission.csv')

# Count the occurrences of each label
label_counts = sentiment_summary['sentiment'].value_counts()

# Print the label counts
print("Label Counts:\n", label_counts)